<a href="https://colab.research.google.com/github/amrutask/Hate_Speech_Detection/blob/master/code/Char_level_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))


Saving cleaned_tweets.csv to cleaned_tweets.csv
User uploaded file "cleaned_tweets.csv" with length 4298346 bytes


In [0]:
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.layers import Input, Embedding, Activation, Flatten, Dense
from keras.layers import Conv1D, MaxPooling1D, Dropout
from keras.models import Model

In [0]:
data=pd.read_csv("cleaned_tweets.csv")

In [144]:
tweets = data.iloc[:,7].values
labels= data.iloc[:,5].values
print(tweets[0:5])
labels[0:5]

['rt as a woman you shouldn t complain about cleaning up your house amp as a man you should always take the trash out'
 'rt boy dats cold tyga dwn bad for cuffin dat hoe in the 1st place'
 'rt dawg rt you ever fuck a bitch and she start to cry you be confused as shit'
 'rt g anderson based she look like a tranny'
 'rt the shit you hear about me might be true or it might be faker than the bitch who told it to ya 57361']


array([2, 1, 1, 1, 1])

In [0]:
from sklearn.model_selection import train_test_split
tweets_train, tweets_test, labels_train, labels_test = train_test_split(tweets, labels, test_size=0.2, random_state=0)

In [146]:
print("Shape of train data : ",tweets_train.shape)
print("Shape of test data : ",tweets_test.shape)

Shape of train data :  (19826,)
Shape of test data :  (4957,)


In [0]:
tk = Tokenizer(num_words=None, char_level=True, oov_token='UNK')
tk.fit_on_texts(tweets_train)

In [148]:
alphabet = "abcdefghijklmnopqrstuvwxyz0123456789"
char_dict = {}
for i, char in enumerate(alphabet):
    char_dict[char] = i + 1
print("Number of unique characters in the model :", len(char_dict))

print(char_dict)

Number of unique characters in the model : 36
{'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, '0': 27, '1': 28, '2': 29, '3': 30, '4': 31, '5': 32, '6': 33, '7': 34, '8': 35, '9': 36}


In [0]:
tk.word_index = char_dict.copy()
# Add 'UNK' to the vocabulary
tk.word_index[tk.oov_token] = max(char_dict.values()) + 1

In [150]:
print(tk.word_index)

train = tk.texts_to_sequences(tweets_train)
test= tk.texts_to_sequences(tweets_test)

print("Size of train data:", len(train))
print("Size of test data:", len(test))

{'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, '0': 27, '1': 28, '2': 29, '3': 30, '4': 31, '5': 32, '6': 33, '7': 34, '8': 35, '9': 36, 'UNK': 37}
Size of train data: 19826
Size of test data: 4957


In [0]:
# Padding
train_data = pad_sequences(train, maxlen=200, padding='post')
test_data = pad_sequences(test, maxlen=200, padding='post')


In [152]:
train_data[0]

array([15,  8, 37, 25,  5,  1,  8, 37,  1, 14,  4, 37,  6, 21,  3, 11, 37,
       20,  8,  5, 37, 12,  1, 11,  5, 18, 19, 37, 20, 15, 15, 37,  2,  9,
       20,  3,  8, 37,  2, 15,  9,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], dtype=int32)

In [0]:
train_data = np.array(train_data, dtype='float32')
test_data = np.array(test_data, dtype='float32')


In [154]:
from keras.utils import to_categorical

train_labels = to_categorical(labels_train)
test_labels = to_categorical(labels_test)

print(train_labels.shape)
print(test_labels.shape)

(19826, 3)
(4957, 3)


In [155]:
vocab_size=len(tk.word_index)
print("Vocabulary size:", vocab_size)

Vocabulary size: 37


In [156]:
embedding_weights = []
embedding_weights.append(np.zeros(vocab_size))

for char, i in tk.word_index.items():
  onehot=np.zeros(vocab_size)
  onehot[i-1]=1
  embedding_weights.append(onehot)
  
embedding_weights = np.array(embedding_weights)

print(embedding_weights.shape)
embedding_weights
  

(38, 37)


array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [0]:
input_size = 200
vocab_size = len(tk.word_index)
embedding_size = 37
conv_layers = [[112, 7, 3],
               [112, 7, 3],
               [112, 3, -1],
               [112, 3, -1],
               [112, 3, -1],
               [112, 3, 3]]

fully_connected_layers = [112, 70]
num_of_classes = 3
dropout_p = 0.25
optimizer = 'adam'
loss = 'categorical_crossentropy'


In [158]:
# Embedding layer Initialization
embedding_layer = Embedding(vocab_size + 1,
                            embedding_size,
                            input_length=input_size,
                            weights=[embedding_weights])

# Model Construction
# Input
inputs = Input(shape=(input_size,), name='input', dtype='int64')  # shape=(?, 1014)
# Embedding
x = embedding_layer(inputs)
# Conv
for filter_num, filter_size, pooling_size in conv_layers:
    x = Conv1D(filter_num, filter_size)(x)
    x = Activation('relu')(x)
    if pooling_size != -1:
        x = MaxPooling1D(pool_size=pooling_size)(x)  # Final shape=(None, 34, 256)
x = Flatten()(x)  # (None, 8704)
# Fully connected layers
for dense_size in fully_connected_layers:
    x = Dense(dense_size, activation='relu')(x)  # dense_size == 1024
    x = Dropout(dropout_p)(x)
# Output Layer
predictions = Dense(num_of_classes, activation='softmax')(x)
# Build model
model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])  # Adam, categorical_crossentropy
model.summary()

# Shuffle
indices = np.arange(train_data.shape[0])
np.random.shuffle(indices)

x_train = train_data[indices]
y_train = train_labels[indices]

x_test = test_data
y_test = test_labels

# Training
model.fit(x_train, y_train,
          validation_data=(x_test, y_test),
          batch_size=128,
          epochs=20)



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 200)               0         
_________________________________________________________________
embedding_8 (Embedding)      (None, 200, 37)           1406      
_________________________________________________________________
conv1d_43 (Conv1D)           (None, 194, 112)          29120     
_________________________________________________________________
activation_42 (Activation)   (None, 194, 112)          0         
_________________________________________________________________
max_pooling1d_21 (MaxPooling (None, 64, 112)           0         
_________________________________________________________________
conv1d_44 (Conv1D)           (None, 58, 112)           87920     
_________________________________________________________________
activation_43 (Activation)   (None, 58, 112)           0         
__________

In [159]:
score = model.evaluate(x_test, y_test, verbose=0) 
print('Test score:', score[0]) 
print('Test accuracy:', score[1])

Test score: 0.5280729737737763
Test accuracy: 0.8916683478391


In [0]:
y_pred= model.predict(x_test)

In [161]:
print(y_pred.shape)
print(y_test.shape)
pred_arr = np.zeros(y_pred.shape[0])
actual_arr = np.zeros(y_test.shape[0])
print(pred_arr.shape)
for i in range(y_pred.shape[0]):
  pred_arr[i]=np.argmax(y_pred[i])
  actual_arr[i]=np.argmax(y_test[i])
print(pred_arr) 
print(actual_arr)


(4957, 3)
(4957, 3)
(4957,)
[2. 1. 1. ... 1. 1. 1.]
[2. 1. 1. ... 1. 1. 1.]


In [162]:
from sklearn.metrics import confusion_matrix

confusion_matrix(actual_arr, pred_arr)

array([[  68,  170,   41],
       [  91, 3619,  142],
       [  22,   71,  733]])